# 📌 ACTIVIDAD 5: REGRESIÓN A PARTIR DE FOTOGRAFÍAS.

## DEFINIR PROBLEMA Y RECOPILAR DATOS

Crea el notebook saa_u03_p01_a5-<tus_iniciales>.ipynb donde entregar esta actividad. Necesitamos
consensuar por votación 2 posibles problemas (lo que escoja la mayoría de la clase gana) más que
nada por obtener suficiente cantidad de datos de alguno de los problemas:

 a) Predecir la edad de una persona: Si nadie está en contra de aportar fotografías personales,
cada alumno buscará 10 fotografías suyas o de conocidos (propias, familia, amigos, …)
realizadas en diferentes edades y las etiquetará con la edad que tenía en ese momento la
persona que aparece "edad_<tus_iniciales>-<num_foto>.jpg" o "edad_<tus_iniciales>
<num_foto>.png". En el caso de descargar de Internet las imágenes o de generarlas con
aplicaciones tened cuidado porque al buscar os pueden aparecer las mismas fotografías para
diferentes edades. La cara debe cubrir casi toda la foto (sin paisaje de fondo: ajustar el borde de
abajo a la barbilla y los laterales a las orejas y el borde superior al pelo) y la persona debe estar
mirando de frente.

 b) Predecir la peligrosidad de un animal en un rango de 0 a 10: 10 significa que te puede
matar o desgraciar si te engancha y 0 que no te va a dañar (al menos en principio). En caso de
escoger esta opción cada uno buscará, procesará y aportará 10 fotografías de cabezas de
animales de todo tipo (serpientes, insectos, felinos, osos, tiburones, ovejas, gatitos, …) con el
nombre del fichero siguiendo el formato "peligo_<tus_iniciales>-<num_foto>.jpg" o bien
formato "peligo_<tus_iniciales>-<num_foto>.png".


Nota: este enfoque no tiene visos de dar buenos resultados. Lo ideal sería extraer característics de cada
fotografía (zonas de ojos, boca, nariz, orejas, dientes) creando embeddings y codificando estos rasgos a
través de deep learning y luego usarlos para realizar las predicciones, pero vamos a probar a ver que tal
nos va, al fin y al cabo es una excusa para probar regresores.

 Una vez que tengas las fotografías debes procesarlas. Te paso el siguiente código:



In [12]:
import numpy as np
import pandas as pd
import fe
import cv2
import os
import re
# ## Instalar opencv: pip install opencv-python

carpeta = "../U03_Recursos/U03_P01_Regresiones/edad/"  # ## CAMBIA!! ## Ruta de carpeta donde están las imágenes
archivo_salida = "samartlop_imagenes.csv"  # ## CAMBIA!! ## Ruta de archivo donde guardar datos como .csv
# Recorrer todas las imágenes en la carpeta
datos_procesados = []
patron = r"^\d+.*\.jpe?g$"  # Para archivos que empiezan con uno o más dígitos y terminan en .jpg o .jpeg
for nombre_archivo in os.listdir("../U03_Recursos/U03_P01_Regresiones/edad/"):
    ruta_completa = os.path.join("../U03_Recursos/U03_P01_Regresiones/edad/", nombre_archivo)
    if not re.match(patron, nombre_archivo):  # Si el fichero no es imagen con dato numérico antes...
        continue
    try:
        edad = int(nombre_archivo.split("_")[0])
    except ValueError:
        print(f"No se pudo extraer la edad del nombre del archivo: {nombre_archivo}")
        continue
    # Leer la imagen en escala de grises, escalarla, guardar la procesada
    imagen = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f"No se pudo leer: {nombre_archivo}")
        continue
    imagen_escalada = cv2.resize(imagen, (92, 112), interpolation=cv2.INTER_AREA) # Escala a 92x112
    cv2.imwrite(os.path.join(carpeta, "img_" + nombre_archivo), imagen_escalada)
    imagen_normalizada = (imagen_escalada / 255.0).astype(np.float32) # Normalizar a [0,1]
    datos_procesados.append({"edad": edad, "imagen": imagen_normalizada}) # Añadir datos a datos_procesados

# Convertir datos_procesados a DataFrame y guardar como csv
df = pd.DataFrame({"edad": [d["edad"] for d in datos_procesados],
                   "imagen": [",".join(map(str, d["imagen"].ravel())) for d in datos_procesados]})
df.to_csv(archivo_salida, index=False)
print(f"✅ Se guardaron {len(datos_procesados)} imágenes procesadas en '{archivo_salida}'")

✅ Se guardaron 0 imágenes procesadas en 'samartlop_imagenes.csv'


 Lo que hace el código es definir rutas (que debes adaptar para tu uso) en las variables carpeta (ruta
relativa para alcanzar el lugar donde están las imágenes) y archivo_salida (pathname relativo que
define el archivo .csv donde se van a guardar los datos).

 Las imágenes de carpeta se transforman usando la librería opencv (quizás debas instalarla) en
información numérica de la siguiente manera: obtendremos una imagen en escala de grises de
dimensiones 92x112 píxels (ancho x alto) que se almacenan como valores float de 32 bits sin signo
entre 0 y 256 normalizados a float en el intervalo [0,1]. La columna target de cada foto será la primera
característica del dataset. Por ejemplo podemos tener ficheros como estos:


 Y al ejecutar el script obtenermos el resultado y generamos:


 Y un fichero .csv donde anotamos los datos de cada ejemplo, algo como esto:


Debes subir tus fotografías debidamente etiquetadas a la siguiente carpeta: 📂 carpeta . Pero antes
comprueba que son correctos y de paso realizas la carga de datos. Para ello prueba a cargar tus
imágenes en un DataFrame de pandas y mostrar sus datos y visualizar alguna de las imágenes que
contiene. El siguiente ejemplo visualiza la primera fotografía usando opencv y aunque sale en su
propia ventana nos sirve para comprobar que las hemos generado y cargado bien:

In [13]:
# Lee el archivo CSV
df = pd.read_csv("samartlop_imagenes.csv")

# Convertir la imagen de string a numpy array
df["imagen"] = df["imagen"].apply(lambda x: np.array(list(map(float, x.split(',')))).reshape(112,92))

# Imprimir datos de la primera cara
print(f"Edad: {df.iloc[0]['Edad']} Imagen: {df.iloc[0]['imagen']}")

pixel_array = (df.iloc[0]["imagen"] * 255).astype(np.uint8)
cv2.imshow("Edad:", pixel_array)
cv2.waitKey(0)
cv2.destroyAllWindows()

IndexError: single positional indexer is out-of-bounds

Este otro código define un método al que indicas (un array de imágenes, un array con sus etiquetas y
desde que imagen hasta qué imagen quieres visualizar). El método usa matplotlib para visualizarlas
añadiendo etiquetas con el dato (la edad en este ejemplo) en un recuadro rojo en la esquina superior
izquierda) y con el índice que ocupa en el DataFrame (una caja de color verde en la esquina inferior
izquierda).

Y aquí está ese otro código:


In [ ]:
import matplotlib.pyplot as plt

def print_imagenes(imgs, targets, desde, hasta):
    # configuramos el tamaño de las imágenes por pulgadas
    fig = plt.figure(figsize=(30, 24))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
    for i in range(desde, hasta):
        # graficamos las imágenes en una matriz de 25x20
        p = fig.add_subplot(25, 20, i + 1, xticks=[], yticks=[])
        p.imshow(imgs[i], cmap="gray")
        # etiquetar imágenes con target e índice
        p.text(0, 14, str(targets[i]), bbox=dict(facecolor='red', alpha=0.5))
        p.text(0, 100, str(i), bbox=dict(facecolor='green', alpha=0.5))
    plt.show()

print_imagenes(df.iloc[:]["imagen"], df.iloc[:]["edad"], 0, 2)

### ENTRENAR VARIOS REGRESORES Y MEDIR SU DESEMPEÑO


Ahora vamos a utilizar varios regresores para ver el desempeño que somos capaces de conseguir en
esta tarea. Debes probar todos los regresores que importamos en esta figura:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

 En primer lugar necesitamos transformar la característica imagen de cada cara en una característica
por cada pixel, para ello:

In [ ]:
# Preparar características del dataframe
y = df['edad']
df_pixels = df["imagen"].apply(lambda img: img.flatten()) # Aplana cada imagen
df_pixels = pd.DataFrame(df_pixels.tolist()) # Expandir en columnas
df_final = pd.concat([df['edad'], df_pixels], axis=1) # Unir con la edad
X = df_final.drop(columns=["edad"]) # Características (píxeles)

Como es algo que haremos en todos los modelos, voy a pasarte el código de un método que nos
ahorrará trabajo. Solo tenemos que pasar en cada llamada los valores y_train, y_test,
y_train_predicho, y_test_predicho y el nombre del modelo. Los valores reales y las predicciones
deben pasarse sin escalar para que se entiendan bien los gráficos. La figura se obtiene con SVR y solo
31 fotos originales:


In [ ]:
def resumen_resultado(y_train, y_train_predicho, y_test, y_test_predicho, nombre_modelo="modelo"):
    rmse_train = mean_squared_error(y_train, y_train_predicho)**0.5
    rmse_test = mean_squared_error(y_test, y_test_predicho)**0.5
    print(f"RMSE en train de ({nombre_modelo}): {rmse_train:.6f}")
    print(f"RMSE en test de ({nombre_modelo}): {rmse_test:.6f}")

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.scatter(y_test, y_test, color="red", label="Valor real")
    plt.scatter(y_test, y_test_predicho, color="blue", label="Valor predicho")
    plt.xlabel("edad")
    plt.ylabel("edad predicha")
    plt.axvline()
    plt.axhline()
    plt.title(f"Valor-predicción en Test ({nombre_modelo})")
    plt.legend()

    plt.subplot(1, 2, 2)
    bar_width = 0.35
    bars1 = plt.bar(2 - bar_width/2, rmse_train, width=bar_width, label="RMSE Train", color="royalblue")
    bars2 = plt.bar(2 + bar_width/2, rmse_test, width=bar_width, label="RMSE Test", color="tomato")

    # Anotar valores encima de las barras
    for bars in [bars1, bars2]:
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval, f'{yval:.2f}', ha='center', va='bottom', fontsize=8)

    plt.ylabel("RMSE")
    plt.title("RMSE en train y test")
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

 ### ENTREGA 9:

a) Añade a la carpeta compartida tus 10 fotografías con el formato indicado.

 b) Adapta el código propuesto, lo entregas y lo ejecutas.

 c) Entrenas y pruebas el modelo
SVR.

 d) Entrenas y pruebas el modelo
DecisionTreeRegressor.

 e) Entrenas y pruebas el modelo
KneighborsRegressor.

 f) Entrenas y pruebas el modelo
BaggingRegressor.

 g) Entrenas y pruebas el modelo
RandomForestRegressor.

 h) Entrenas y pruebas el modelo
AdaBoostRegressor.

 i)  Entrenas y pruebas el modelo
 GradientBoostingRegressor.
 underfitting

 j) Comenta en cada uno de ellos los ajustes de hiperparámetros que has intentado para evitar
que tengan
o bien
overfitting
 . Si no consigues solucionarlo deja la mejor
configuración